# Pré-processamento para interpolação de voxel
* Todos os conjuntos de dados foram pré-processados para obter o tamanho de voxel isométricos de (1x1x1). Isso foi feito usando a biblioteca SimpleITK, pois esta permite mudar o tamanho do voxel em precisar manupular manualmente a affine dos volumes. Essa biblioteca tem algumas funções que faz isso automaticamente, garantindo a compatibilidade dos dados para o treinamento. As imagens e as segmentações passaram por esse pré-processamento. Alguns volumes dos conjuntos de dados do CC359 já tem os voxels isométricos e um desses foi usado como referência para aplicar nos outros. 

In [1]:
from scipy.ndimage import zoom
import SimpleITK as sitk
import nibabel as nib
import subprocess
import numpy as np
import glob
import os

def save_nii_zoom(input_paths: str, ref_path: str):
    '''
    Documentação
    '''
    for input_path in input_paths:
        T1_file = 'brain_T1w.nii.gz'
        sitk_T1 = sitk.ReadImage(os.path.join(input_path, T1_file))
        origin = sitk_T1.GetOrigin()
        spacing = sitk_T1.GetSpacing()
        directions = sitk_T1.GetDirection()

        mask_T1_file = 'brainmask_T1w.nii.gz'
        sitk_mask_T1 = sitk.ReadImage(os.path.join(input_path, mask_T1_file))

        pre_affine = nib.load(os.path.join(input_path, T1_file)).affine
        print(f"Pre-affine {pre_affine}")  

        T1_array = sitk.GetArrayFromImage(sitk_T1)
        mask_T1_array = sitk.GetArrayFromImage(sitk_mask_T1)

        input_spacing = np.array(spacing[::-1])  # 1, 1, 2 -> 2, 1, 1
        target_spacing = np.array(sitk.ReadImage(ref_path).GetSpacing())[::-1]
        print(f"Transforming spacing of {T1_array.shape} from {input_spacing} to {target_spacing}")

        T1_iso_array = zoom(T1_array, input_spacing/target_spacing)
        mask_T1_iso_array = zoom(mask_T1_array, input_spacing/target_spacing, order=0)
        print(f"New shape: {T1_iso_array.shape}")

        iso_T1 = sitk.GetImageFromArray(T1_iso_array)
        iso_T1.SetSpacing(target_spacing[::-1])
        iso_T1.SetOrigin(origin)
        iso_T1.SetDirection(directions)
        iso_mask_T1 = sitk.GetImageFromArray(mask_T1_iso_array)
        iso_mask_T1.SetSpacing(target_spacing[::-1])
        iso_mask_T1.SetOrigin(origin)
        iso_mask_T1.SetDirection(directions)



        T1_out_name = "iso_brain_T1w.nii.gz"
        mask_T1_out_name = "iso_brainmask_T1w.nii.gz"

        sitk.WriteImage(iso_T1, os.path.join(input_path, T1_out_name))
        sitk.WriteImage(iso_mask_T1, os.path.join(input_path, mask_T1_out_name))
        post_affine = nib.load(os.path.join(input_path, T1_out_name)).affine
        print(f"Post-affine {post_affine}")

        print("Done")

    return iso_T1, iso_mask_T1

In [3]:
ref_path = "/home/joany/dataset_train/val/CC0126/brain_T1w.nii.gz"
folder_mri = f"/home/joany/dataset_train/train"
input_paths = glob.glob(os.path.join(folder_mri, "*"))

image, label = save_nii_zoom(input_paths, ref_path)

Pre-affine [[ 9.99160826e-01 -2.69156182e-03  3.62634324e-02 -7.33831024e+01]
 [ 2.75593041e-03  8.88865411e-01  5.97652514e-03 -1.22508713e+02]
 [-4.08155620e-02 -5.87158231e-03  8.88128757e-01 -1.26126587e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Transforming spacing of (288, 288, 150) from [0.8888889  0.8888889  0.99999797] to [1. 1. 1.]
New shape: (256, 256, 150)
Post-affine [[ 9.99162912e-01 -3.02800699e-03  4.07963619e-02 -7.33831024e+01]
 [ 2.75593600e-03  9.99973595e-01  6.72359066e-03 -1.22508713e+02]
 [-4.08156440e-02 -6.60553016e-03  9.99144852e-01 -1.26126587e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Done
Pre-affine [[ 9.99643505e-01 -2.65690163e-02  2.73752026e-03 -1.25807167e+02]
 [ 2.65813135e-02  9.99981284e-01 -8.38861056e-03 -1.06818146e+02]
 [-2.51368107e-03  8.46151635e-03  9.99961138e-01 -1.42651108e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Transforming spacing of (256, 196